In [ ]:
import duckdb
import pandas as pd

In [ ]:
import duckdb
import pandas as pd
con = duckdb.connect("fno_trading.db")

In [ ]:
con.execute("""
CREATE TABLE raw_trades AS
SELECT *
FROM read_csv_auto('../Data/NSE_FO.csv');
""")

In [ ]:
con.execute("SELECT * FROM raw_trades").df()

In [ ]:
con.execute("""
CREATE TABLE exchanges (
    exchange_id INTEGER PRIMARY KEY,
    exchange_code VARCHAR
);
""")

INSERT_EXCHANGE = """
INSERT INTO exchanges VALUES
(1, 'NSE'),
(2, 'BSE'),
(3, 'MCX');
"""
con.execute(INSERT_EXCHANGE)

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE instruments AS
SELECT DISTINCT
    ROW_NUMBER() OVER () AS instrument_id,
    INSTRUMENT,
    SYMBOL
FROM raw_trades;
""")

In [ ]:
con.execute("""
CREATE TABLE expiries AS
SELECT DISTINCT
    ROW_NUMBER() OVER () AS expiry_id,
    STRPTIME(EXPIRY_DT, '%d-%b-%Y') AS expiry_dt,
    STRIKE_PR,
    OPTION_TYP
FROM raw_trades;
""")

In [ ]:
con.execute("""
CREATE TABLE tmp_trades AS
SELECT
  i.instrument_id,
  1 AS exchange_id,
  e.expiry_id,
  e.option_typ,
  OPEN,
  HIGH,
  LOW,
  CLOSE,
  SETTLE_PR,
  CONTRACTS,
  VAL_INLAKH,
  OPEN_INT,
  CHG_IN_OI,
  STRPTIME(r.TIMESTAMP, '%d-%b-%Y') + INTERVAL '15:30:00' AS trade_timestamp
FROM raw_trades r
JOIN instruments i
  ON r.INSTRUMENT = i.INSTRUMENT
 AND r.SYMBOL = i.SYMBOL
JOIN expiries e
  ON STRPTIME(r.EXPIRY_DT, '%d-%b-%Y') = e.expiry_dt
 AND r.STRIKE_PR = e.STRIKE_PR
 AND r.OPTION_TYP = e.OPTION_TYP
 """)

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE instruments AS
SELECT DISTINCT
    ROW_NUMBER() OVER () AS instrument_id,
    INSTRUMENT,
    SYMBOL
FROM raw_trades;
""")

In [ ]:
con.execute("""CREATE INDEX idx_trades_timestamp ON trades(trade_timestamp);
CREATE INDEX idx_instruments_symbol ON instruments(symbol);
CREATE INDEX idx_trades_exchange ON trades(exchange_id);
""")

#### Top 10 instruments with the highest increase in open interest

In [ ]:
query = """
SELECT
    i.symbol,
    SUM(t.chg_in_oi) AS total_oi_change
FROM trades t
JOIN instruments i ON t.instrument_id = i.instrument_id
GROUP BY i.symbol
ORDER BY total_oi_change DESC
LIMIT 10;
"""
con.execute(query).df()

#### Rolling 7-day volatility of closing prices for the instrument 'NIFTY'

In [ ]:
query = """SELECT
  t.trade_timestamp,
  STDDEV(t.close) OVER (
    ORDER BY t.trade_timestamp
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS rolling_volatility
FROM trades t
JOIN instruments i ON t.instrument_id = i.instrument_id
WHERE i.symbol = 'NIFTY';
"""
con.execute(query).df()

#### Average settlement price per exchange

In [ ]:
query = """
SELECT
  e.exchange_code,
  AVG(t.settle_pr) AS avg_settle_price
FROM trades t
JOIN exchanges e ON t.exchange_id = e.exchange_id
GROUP BY e.exchange_code;
"""
con.execute(query).df()

#### Total traded volume per expiry date and strike price

In [ ]:
query = """SELECT
  e.expiry_dt,
  e.strike_pr,
  SUM(t.contracts) AS total_volume
FROM trades t
JOIN expiries e ON t.expiry_id = e.expiry_id
GROUP BY e.expiry_dt, e.strike_pr
ORDER BY e.expiry_dt, e.strike_pr;
"""
con.execute(query).df()

#### Top 10 trades with the highest number of contracts in the last 30 days 

In [ ]:
query = """
SELECT *
FROM trades
WHERE trade_timestamp >= (SELECT MAX(trade_timestamp) - INTERVAL '30 days' FROM trades)
ORDER BY contracts DESC
LIMIT 10;
"""
con.execute(query).df()

#### Query plan for retrieving trades in the last 30 days

In [ ]:
EXPLAIN ANALYZE
SELECT *
FROM trades
WHERE trade_timestamp >= (SELECT MAX(trade_timestamp) - INTERVAL '30 days' FROM trades);